In [4]:
import numpy as np
np.__version__
import math

In [37]:
class TicTacToe:
    #def __init__(self):
        #pass
    def __init__(self):
        self.rowCount=3
        self.columnCount=3
        self.actualSize=self.rowCount*self.columnCount
        
    def initial_state(self):
        return np.zeros((self.rowCount,self.columnCount))
    
    def get_valid_moves(self,state):
        return (state.reshape(-1)==0).astype(np.uint8)
    
    def get_next_state(self,state,action,player):
        row=action//self.rowCount
        column=action%self.columnCount
        state[row,column]=player
        return state
    
    
    def check_win(self,state,action):
        row=action//self.rowCount
        column=action%self.columnCount
        player=state[row,column]
        
        return(
            np.sum(state[row, :])==player*self.rowCount
            or np.sum(state[:,column])==player*self.columnCount 
            or np.sum(np.diag(state))==player*self.columnCount
            or np.sum(np.diag(np.flip(state,axis=0)))==player*self.columnCount
        )
            
            
            
    def getValue_terminate(self,state,action):
        if self.check_win(state,action):
            return 1,True
        elif np.sum(self.get_valid_moves(state))==0:
            return 0,True
        return 0,False
            
    def player_change(self,player):
        return -player
    def get_opponent(self,player):
        return -player
    def get_opponent_status(self,value):
        return -value
    def change_perspective(self,state,player):
        return state*player

In [ ]:
class Node:
    def __init(self,state,args,tttObj,parent=None,action_taken=None):
        self.state=state
        self.tttObj=tttObj
        self.parent=parent
        self.action_taken=action_taken
        self.args=args
        
        self.children=[]
        self.expandable=tttObj.get_valid_moves(state)
        self.viewCount=0
        self.valueSum=0
        
    def fully_expanded(self,state):
        return np.sum(self.expandable)==0 and len(self.children)>0
    
    def select(self):
        best_child=None
        best_ucb=np.inf
        for child in self.children:
            ucb=self.get_ucb(child)
            if ucb>best_ucb:
                best_ucb=ucb
                best_child=child
        return best_child
            
    def get_ucb(self,child):
        q_value=1-((child.valueSum/child.viewCount)-1)/2
        return q_value+self.args['C']*math.sqrt(math.log(self.viewCount)/child.valueCount)
    
    def expand(self,state):
        action=np.random.choice(np.where(self.expandable==1)[0])
        self.expandable(action)=0
        
        child_state=self.state.copy()
        child_state=self.tttObj.get_next_state(child_state,action,1)
        child_state=self.tttObj.change_persepective(child_state,player=1)
        
        child=Node(child_state,self.args,self.tttObj,self,action)
        self.children.append(child)
        
        return child
        
    def simulate(self,state):
        value,termination=self.tttObj.getValue_terminate(self.state,self.action_taken)
        value=self.tttObj.get_opponent_status(value)
        rollout_state=self.state.copy()
        rollout_player=1
        while True:
            validMoves=self.tttObj.get_valid_moves(rollout_state)
            action=np.random.choice(np.where(validMoves==1)[0])
            rollout_state=self.tttObj.get_next_state(rollout_state,action,rollout_player)
            value,termination=self.tttObj.getValue_terminate(rollout_state,action)
            
            
    def backpropogate(self):
        self.viewCount=self.viewCount+1
        self.valueSum=self.valueSum
        
        
            
                    
                
        
        
    
class MCTS:
    def __init__(self,tttObj,args):
        #tttObj is the obj for class TicTacToe
        self.tttObj=tttObj
        self.args=args
    
    def search(self,state):
        root=Node(state,args,tttObj)
        for search in range(self.args['num_searches']):
            node=root
            
            while node.fully_expanded:
                node=node.select
            value,termination=self.tttObj.getValue_terminate(node.state,node.action_taken)
            value=self.tttObj.get_opponent_status(value)
            if not termination:
                node=node.expand() #expansion
                node=node.simulate()
        

In [38]:
tictactoe=TicTacToe()
player=1
state=tictactoe.initial_state()
while(True):
    print(state)
    valid_moves=tictactoe.get_valid_moves(state)
    print("valid moves are",[i for i in range(tictactoe.actualSize) if valid_moves[i]==1])
    
        #print(i)
    action=int(input(f"{player}"))
    
    if valid_moves[action]==0:
        print("action not valid")
        continue
        
    state=tictactoe.get_next_state(state,action,player)
    value,terminate=tictactoe.getValue_terminate(state,action)
    
    if terminate:
        print(state)
        if value==1:
            print(f"{player} won")
        else:
            print("Draw")
        break
    player=tictactoe.player_change(player)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
valid moves are [0, 1, 2, 3, 4, 5, 6, 7, 8]
12
[[0. 0. 1.]
 [0. 0. 0.]
 [0. 0. 0.]]
valid moves are [0, 1, 3, 4, 5, 6, 7, 8]
-11
[[ 0. -1.  1.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
valid moves are [0, 3, 4, 5, 6, 7, 8]
15
[[ 0. -1.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  0.]]
valid moves are [0, 3, 4, 6, 7, 8]
-17
[[ 0. -1.  1.]
 [ 0.  0.  1.]
 [ 0. -1.  0.]]
valid moves are [0, 3, 4, 6, 8]
18
[[ 0. -1.  1.]
 [ 0.  0.  1.]
 [ 0. -1.  1.]]
1 won
